In [1]:
import sys
import os
os.environ["NEMO_CACHE_DIR"] = "/your/desired/path"
sys.path.insert(0, '/usr/src/app/inference')

import logging
logging.basicConfig(level = logging.INFO)
logging.getLogger("nemo_logger").setLevel(logging.ERROR)

In [2]:
from canary_transcriber import CanaryTranscriber
from force_aligner import ForceAligner
from diarizer import Diarizer

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0209 07:26:35.806730 140105438622848 zarr.py:57] `zarr` distributed checkpoint backend is deprecated. Please switch to PyTorch Distributed format (`torch_dist`).


In [3]:
transcriber = CanaryTranscriber(model_name='nvidia/canary-1b')

[NeMo I 2025-02-09 07:26:41 mixins:197] _setup_tokenizer: detected an aggregate tokenizer
[NeMo I 2025-02-09 07:26:41 mixins:336] Tokenizer SentencePieceTokenizer initialized with 32 tokens
[NeMo I 2025-02-09 07:26:41 mixins:336] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2025-02-09 07:26:41 mixins:336] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2025-02-09 07:26:41 mixins:336] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2025-02-09 07:26:41 mixins:336] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2025-02-09 07:26:41 aggregate_tokenizer:73] Aggregate vocab size: 4128


[NeMo W 2025-02-09 07:26:42 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    batch_size: null
    num_workers: 8
    use_lhotse: true
    max_duration: 40
    pin_memory: true
    use_bucketing: false
    bucket_duration_bins: null
    num_buckets: 1
    text_field: answer
    lang_field: target_lang
    batch_duration: 360
    quadratic_duration: 15
    bucket_buffer_size: 20000
    shuffle_buffer_size: 10000
    
[NeMo W 2025-02-09 07:26:42 modelPT:183] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 

[NeMo I 2025-02-09 07:26:42 features:305] PADDING: 0
[NeMo I 2025-02-09 07:26:53 save_restore_connector:272] Model EncDecMultiTaskModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--canary-1b/snapshots/dd32c0c709e2bfc79f583e16b9df4b3a160f7e86/canary-1b.nemo.
[NeMo I 2025-02-09 07:26:53 aed_multitask_models:260] Changed decoding strategy to 
    strategy: beam
    compute_hypothesis_token_set: false
    preserve_alignments: null
    compute_langs: false
    beam:
      beam_size: 1
      search_type: default
      len_pen: 1.0
      max_generation_delta: 20
      return_best_hypothesis: true
      preserve_alignments: false
    temperature: 1.0
    


In [4]:
force_aligner = ForceAligner(model_name=None)

In [5]:
from pydub import AudioSegment

def split_audio(audio_filepath, chunk_duration_sec=600):
	"""
	Splits the audio into chunks of up to `chunk_duration_sec`.
	Returns a list of tuples (chunk_file_path, chunk_duration).

	We use pydub for splitting. Each chunk is saved as a temporary .wav.
	"""
	audio_seg = AudioSegment.from_file(audio_filepath)
	total_ms = len(audio_seg)
	chunk_ms = chunk_duration_sec * 1000

	chunks = []
	start_ms = 0
	idx = 0

	while start_ms < total_ms:
		end_ms = min(start_ms + chunk_ms, total_ms)
		chunk = audio_seg[start_ms:end_ms]
		chunk_duration = (end_ms - start_ms) / 1000.0

		chunk_path = f"{audio_filepath}_chunk_{idx}.wav"
		chunk.export(chunk_path, format="wav")
		chunks.append((chunk_path, chunk_duration))

		start_ms += chunk_ms
		idx += 1

	return chunks

def parse_ctm(ctm_file):
	items = []
	with open(ctm_file, "r", encoding="utf-8") as f:
		for line in f:
			parts = line.strip().split()
			if len(parts) < 5:
				continue
			start = float(parts[2])
			dur = float(parts[3])
			txt = parts[4]
			items.append({
				"start": round(start, 3),
				"duration": round(dur, 3),
				"end": round(start + dur, 3),
				"text": txt
			})
	return items


def parse_alignment(alignment_result):
	"""
	Reuses your standard CTM parsing logic, returning a dict with
	text, words, tokens, segments.
	"""
	alignment_data = {
		"text": alignment_result.get("text", ""),
		"words": [],
		"tokens": [],
		"segments": []
	}
	w_ctm = alignment_result.get("words_level_ctm_filepath")
	t_ctm = alignment_result.get("tokens_level_ctm_filepath")
	s_ctm = alignment_result.get("segments_level_ctm_filepath")

	if w_ctm and os.path.exists(w_ctm):
		alignment_data["words"] = parse_ctm(w_ctm)
	if t_ctm and os.path.exists(t_ctm):
		alignment_data["tokens"] = parse_ctm(t_ctm)
	if s_ctm and os.path.exists(s_ctm):
		alignment_data["segments"] = parse_ctm(s_ctm)
	return alignment_data

def shift_alignment(alignment_data, offset_sec):
	"""
	Shifts the start/end times in alignment_data by offset_sec, in place.
	This is useful when concatenating chunk alignments into a single timeline.
	"""
	for item in alignment_data["words"]:
		item["start"] += offset_sec
		item["end"] += offset_sec
	for item in alignment_data["tokens"]:
		item["start"] += offset_sec
		item["end"] += offset_sec
	for item in alignment_data["segments"]:
		item["start"] += offset_sec
		item["end"] += offset_sec


In [6]:
audio_filepath = '/usr/src/app/mono_output.wav'
chunk_duration_sec = 10

chunk_paths = split_audio(audio_filepath, chunk_duration_sec)
all_words = []
all_tokens = []
all_segments = []
offset_sec = 0.0
final_transcript_parts = []

for chunk_file, chunk_dur in chunk_paths:
	# 1. Transcription on chunk
	chunk_text = transcriber.transcribe(chunk_file)
	final_transcript_parts.append(chunk_text)

	# 2. Force align chunk
	chunk_alignment_result = force_aligner.align(chunk_file, chunk_text)
	# Parse alignment into memory
	chunk_alignment_data = parse_alignment(chunk_alignment_result)

	# Adjust times by offset
	shift_alignment(chunk_alignment_data, offset_sec)

	# Merge into global arrays
	all_words.extend(chunk_alignment_data["words"])
	all_tokens.extend(chunk_alignment_data["tokens"])
	all_segments.extend(chunk_alignment_data["segments"])

	offset_sec += chunk_dur

	# Clean up chunk temp file if needed
	os.remove(chunk_file)

final_transcript = " ".join(final_transcript_parts).strip()
merged_alignment = {
	"text": final_transcript,
	"words": all_words,
	"tokens": all_tokens,
	"segments": all_segments,
}

Transcribing: 1it [00:01,  1.90s/it]

[NeMo I 2025-02-09 07:26:55 align:159] Hydra config: pretrained_name: stt_en_fastconformer_hybrid_large_pc
    model_path: null
    manifest_filepath: /tmp/nfa_08jf7ip6/ec50fe5d-2596-47c3-93bf-33d40f5bdf91_manifest.json
    output_dir: /tmp/nfa_08jf7ip6/nfa_output
    align_using_pred_text: false
    transcribe_device: null
    viterbi_device: null
    batch_size: 1
    use_local_attention: true
    additional_segment_grouping_separator: '|'
    audio_filepath_parts_in_utt_id: 1
    use_buffered_chunked_streaming: false
    chunk_len_in_secs: 1.6
    total_buffer_in_secs: 4.0
    chunk_batch_size: 32
    simulate_cache_aware_streaming: false
    save_output_file_formats:
    - ctm
    ctm_file_config:
      remove_blank_tokens: false
      minimum_timestamp_duration: 0.0
    ass_file_config:
      fontsize: 20
      vertical_alignment: center
      resegment_text_to_fill_space: false
      max_lines_per_segment: 2
      text_already_spoken_rgb:
      - 49
      - 46
      - 61
      te


[NeMo W 2025-02-09 07:26:55 align:237] One or both of transcribe_device and viterbi_device are GPUs. If you run into OOM errors it may help to change both devices to be the CPU.


[NeMo I 2025-02-09 07:26:55 cloud:58] Found existing object /your/desired/path/stt_en_fastconformer_hybrid_large_pc/465b32000fc320f5905fda11a1866ef6/stt_en_fastconformer_hybrid_large_pc.nemo.
[NeMo I 2025-02-09 07:26:55 cloud:64] Re-using file from: /your/desired/path/stt_en_fastconformer_hybrid_large_pc/465b32000fc320f5905fda11a1866ef6/stt_en_fastconformer_hybrid_large_pc.nemo
[NeMo I 2025-02-09 07:26:55 common:826] Instantiating model from pre-trained checkpoint
[NeMo I 2025-02-09 07:26:56 mixins:173] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2025-02-09 07:26:58 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    is_concat: false
    concat_sampling_technique: random
    concat_sampling_temperature: 8
    concat_shuffle: true
    concat_sampling_scale: 1.0
    manifest_filepath:
    - - /data/mml/en/tarred_train/pcstrip_sharded_manifests/manifest__OP_0..511_CL_.json
    tarred_audio_filepaths:
    - - /data/mml/en/tarred_train/audio__OP_0..511_CL_.tar
    concat_sampling_probabilities:
    - 1.0
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20
    min_duration: 0.1
    is_tarred: true
    shard_manifests: true
    defer_setup: true
    shuffle_n: 528387
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[NeMo W 20

...

[NeMo I 2025-02-09 07:27:13 cloud:58] Found existing object /your/desired/path/stt_en_fastconformer_hybrid_large_pc/465b32000fc320f5905fda11a1866ef6/stt_en_fastconformer_hybrid_large_pc.nemo.
[NeMo I 2025-02-09 07:27:13 cloud:64] Re-using file from: /your/desired/path/stt_en_fastconformer_hybrid_large_pc/465b32000fc320f5905fda11a1866ef6/stt_en_fastconformer_hybrid_large_pc.nemo
[NeMo I 2025-02-09 07:27:13 common:826] Instantiating model from pre-trained checkpoint
[NeMo I 2025-02-09 07:27:14 mixins:173] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2025-02-09 07:27:15 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    is_concat: false
    concat_sampling_technique: random
    concat_sampling_temperature: 8
    concat_shuffle: true
    concat_sampling_scale: 1.0
    manifest_filepath:
    - - /data/mml/en/tarred_train/pcstrip_sharded_manifests/manifest__OP_0..511_CL_.json
    tarred_audio_filepaths:
    - - /data/mml/en/tarred_train/audio__OP_0..511_CL_.tar
    concat_sampling_probabilities:
    - 1.0
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20
    min_duration: 0.1
    is_tarred: true
    shard_manifests: true
    defer_setup: true
    shuffle_n: 528387
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[NeMo W 20

[NeMo I 2025-02-09 07:27:15 features:305] PADDING: 0
[NeMo I 2025-02-09 07:27:16 rnnt_models:224] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-02-09 07:27:16 rnnt_models:224] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-02-09 07:27:16 rnnt_models:224] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-02-09 07:27:16 save_restore_connector:272] Model EncDecHybridRNNTCTCBPEModel was successfully restored from /your/desired/path/stt_en_fastconformer_hybrid_large_pc/465b32000fc320f5905fda11a1866ef6/stt_en_fastconformer_hybrid_large_pc.nemo.
[NeMo I 2025-02-09 07:27:16 hybrid_rnnt_ctc_bpe_models:464] No `decoding_cfg` passed when changing decoding strategy, using internal config
[NeMo I 2025-02-09 07:27:16 hybrid_rnnt_ctc_bpe_models:488] Changed decoding strategy of the CTC decoder

Transcribing: 100%|██████████| 1/1 [00:00<00:00, 10.35it/s]

[NeMo I 2025-02-09 07:27:17 data_prep:822] Calculated that the model downsample factor is 8 and therefore the ASR model output timestep duration is 0.08 -- will use this for all batches


In [7]:
print(merged_alignment)
alignment_result_new = {
	"audio_filepath": audio_filepath,
	"text": merged_alignment['text'],
	"words_level_ctm_filepath": None,  # Not strictly needed by diarizer
	"tokens_level_ctm_filepath": None,
	"segments_level_ctm_filepath": None,
}

{'text': "Ah, now Rich, would you like some pussy? Well, it wasn't on my mind right now. It is now. Pussy energy drink. What flavor is it? Flavor. Dream it. Yeah, moving on. And I'd like to introduce something for which at first I thought I'm going to struggle to find a motoring application because what it is is this. This machine is controlled by your iPhone, right? With an app, and it flies up in the air, and there's a camera on it.", 'words': [{'start': 0.32, 'duration': 0.48, 'end': 0.8, 'text': 'Ah,'}, {'start': 0.88, 'duration': 0.08, 'end': 0.96, 'text': 'now'}, {'start': 1.28, 'duration': 0.56, 'end': 1.84, 'text': 'Rich,'}, {'start': 1.92, 'duration': 0.08, 'end': 2.0, 'text': 'would'}, {'start': 2.32, 'duration': 0.08, 'end': 2.4, 'text': 'you'}, {'start': 2.56, 'duration': 0.08, 'end': 2.64, 'text': 'like'}, {'start': 2.8, 'duration': 0.08, 'end': 2.88, 'text': 'some'}, {'start': 3.04, 'duration': 2.32, 'end': 5.36, 'text': 'pussy?'}, {'start': 5.92, 'duration': 0.32, 'end':

In [9]:
diarizer_instance_clust = Diarizer(num_speakers=None, use_oracle_vad=False, out_dir="diarizer_output_clust")
diarizer_instance_clust.diarize(alignment_result_new)

[NeMo W 2025-02-09 07:28:19 model_utils:492] Skipped conversion for config/subconfig:
    {'manifest_filepath': '???', 'out_dir': 'diarizer_output_clust', 'oracle_vad': False, 'collar': 0.25, 'ignore_overlap': True, 'vad': {'model_path': 'vad_multilingual_marblenet', 'external_vad_manifest': None, 'parameters': {'window_length_in_sec': 0.63, 'shift_length_in_sec': 0.01, 'smoothing': False, 'overlap': 0.5, 'onset': 0.8, 'offset': 0.6, 'pad_onset': 0, 'pad_offset': -0.05, 'min_duration_on': 0, 'min_duration_off': 0.6, 'filter_speech_first': True}}, 'speaker_embeddings': {'model_path': 'titanet_large', 'parameters': {'window_length_in_sec': [1.5, 1.25, 1.0, 0.75, 0.5], 'shift_length_in_sec': [0.75, 0.625, 0.5, 0.375, 0.1], 'multiscale_weights': [1, 1, 1, 1, 1], 'save_embeddings': True}}, 'clustering': {'parameters': {'oracle_num_speakers': False, 'max_num_speakers': 8, 'enhanced_count_thres': 80, 'max_rp_threshold': 0.25, 'sparse_search_volume': 30, 'maj_vote_spk_count': False, 'chunk_clu

[NeMo I 2025-02-09 07:28:19 clustering_diarizer:127] Loading pretrained vad_multilingual_marblenet model from NGC
[NeMo I 2025-02-09 07:28:19 cloud:58] Found existing object /your/desired/path/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.
[NeMo I 2025-02-09 07:28:19 cloud:64] Re-using file from: /your/desired/path/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo
[NeMo I 2025-02-09 07:28:19 common:826] Instantiating model from pre-trained checkpoint


[NeMo W 2025-02-09 07:28:19 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/ami_train_0.63.json,/manifests/freesound_background_train.json,/manifests/freesound_laughter_train.json,/manifests/fisher_2004_background.json,/manifests/fisher_2004_speech_sampled.json,/manifests/google_train_manifest.json,/manifests/icsi_all_0.63.json,/manifests/musan_freesound_train.json,/manifests/musan_music_train.json,/manifests/musan_soundbible_train.json,/manifests/mandarin_train_sample.json,/manifests/german_train_sample.json,/manifests/spanish_train_sample.json,/manifests/french_train_sample.json,/manifests/russian_train_sample.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 256
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: sca

[NeMo I 2025-02-09 07:28:19 features:305] PADDING: 16
[NeMo I 2025-02-09 07:28:19 save_restore_connector:272] Model EncDecClassificationModel was successfully restored from /your/desired/path/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.
[NeMo I 2025-02-09 07:28:19 clustering_diarizer:160] Loading pretrained titanet_large model from NGC
[NeMo I 2025-02-09 07:28:19 cloud:58] Found existing object /your/desired/path/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.
[NeMo I 2025-02-09 07:28:19 cloud:64] Re-using file from: /your/desired/path/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo
[NeMo I 2025-02-09 07:28:19 common:826] Instantiating model from pre-trained checkpoint


[NeMo W 2025-02-09 07:28:19 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2025-02-09 07:28:19 modelPT:183] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

...


clustering: 100%|██████████| 1/1 [00:00<00:00,  2.41it/s]

[NeMo I 2025-02-09 07:28:25 clustering_diarizer:461] Outputs are saved in /usr/src/app/tests/diarizer_output_clust directory



[NeMo W 2025-02-09 07:28:25 der:185] Check if each ground truth RTTMs were present in the provided manifest file. Skipping calculation of Diariazation Error Rate


Diarization complete. Predicted RTTM at: diarizer_output_clust/pred_rttms/mono_output.rttm


'diarizer_output_clust/pred_rttms/mono_output.rttm'